In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import findspark
findspark.init()

/tmp/ipykernel_10111/4280649393.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [1]:
from pyspark.sql import SparkSession
import os

os.environ["SPARK_LOCAL_IP"] = "127.0.0.1"

spark = SparkSession.builder \
    .appName("USPS") \
    .config("spark.executor.memory", "8G") \
    .config("spark.driver.memory", "8G") \
    .config("spark.executor.cores", "4") \
    .config("spark.driver.maxResultSize", "4G") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")


/tmp/ipykernel_65386/1922623038.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/21 21:14:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/21 21:14:01 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/06/21 21:14:01 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/06/21 21:14:01 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/06/21 21:14:01 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
24/06/21 21:14:01 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
24/06/21 21:14:01 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.


In [3]:
df = spark.read.csv("/home/pk/DAEN690/package_data/PKG_Origin_PcLevel.txt", header=True, sep="|")

In [4]:
df.show(3)

+-----------+---------------+--------------------+----------------+---------------------+-------------------+-------------+
|MailPieceID|ServiceTypeCode|StartTheClockZipCode|StopTheClockDate|ScheduledDeliveryDate|StopTheClockZipCode|MailClassCode|
+-----------+---------------+--------------------+----------------+---------------------+-------------------+-------------+
|88228305910|           001 |               37040|       1/16/2024|            1/16/2024|              15613|           G0|
|88180422391|           001 |               37090|       1/16/2024|            1/16/2024|              92247|           G0|
|88147885963|           055 |               37167|       1/11/2024|            1/11/2024|              33770|           PM|
+-----------+---------------+--------------------+----------------+---------------------+-------------------+-------------+
only showing top 3 rows



In [7]:
import pandas as pd
pandas_df = df.limit(3).toPandas()
pandas_df

,MailPieceID,ServiceTypeCode,StartTheClockZipCode,StopTheClockDate,ScheduledDeliveryDate,StopTheClockZipCode,MailClassCode
0,88228305910,001,37040,1/16/2024,1/16/2024,15613,G0
1,88180422391,001,37090,1/16/2024,1/16/2024,92247,G0
2,88147885963,055,37167,1/11/2024,1/11/2024,33770,PM


In [4]:
df.count()

1115279

In [5]:
df.printSchema()

root
 |-- MailPieceID: string (nullable = true)
 |-- ServiceTypeCode: string (nullable = true)
 |-- StartTheClockZipCode: string (nullable = true)
 |-- StopTheClockDate: string (nullable = true)
 |-- ScheduledDeliveryDate: string (nullable = true)
 |-- StopTheClockZipCode: string (nullable = true)
 |-- MailClassCode: string (nullable = true)



In [6]:
from pyspark.sql.functions import to_date

df = df.withColumn("StopTheClockDate", to_date("StopTheClockDate", "M/d/yyyy"))
df = df.withColumn("ScheduledDeliveryDate", to_date("ScheduledDeliveryDate", "M/d/yyyy"))

df.show(3)
df.printSchema()

+-----------+---------------+--------------------+----------------+---------------------+-------------------+-------------+
|MailPieceID|ServiceTypeCode|StartTheClockZipCode|StopTheClockDate|ScheduledDeliveryDate|StopTheClockZipCode|MailClassCode|
+-----------+---------------+--------------------+----------------+---------------------+-------------------+-------------+
|88228305910|           001 |               37040|      2024-01-16|           2024-01-16|              15613|           G0|
|88180422391|           001 |               37090|      2024-01-16|           2024-01-16|              92247|           G0|
|88147885963|           055 |               37167|      2024-01-11|           2024-01-11|              33770|           PM|
+-----------+---------------+--------------------+----------------+---------------------+-------------------+-------------+
only showing top 3 rows

root
 |-- MailPieceID: string (nullable = true)
 |-- ServiceTypeCode: string (nullable = true)
 |-- StartTh

In [8]:
df2 = spark.read.parquet("/home/pk/DAEN690/mail_data/Scan Output/result.parquet")
df2.show(3)

+-----------+-------------------+-------------------+-----------+------------------+--------------------+------------------+
|MailPieceID|  min_scan_datetime|  max_scan_datetime|total_scans|Distinct_zip_scans|Distinct_event_scans|time_delta_minutes|
+-----------+-------------------+-------------------+-----------+------------------+--------------------+------------------+
|88275302703|2024-01-14 23:05:28|2024-01-22 16:26:24|         27|                 9|                  14|             11520|
|88246618470|2024-01-12 15:09:48|2024-01-15 11:11:41|         27|                 8|                  13|              4320|
|88365344486|2024-01-18 09:29:59|2024-01-23 15:38:41|         31|                 8|                  15|              7200|
+-----------+-------------------+-------------------+-----------+------------------+--------------------+------------------+
only showing top 3 rows



In [9]:
# Join DataFrames on MailPieceID
joined_df = df.join(df2, "MailPieceID", "inner")

In [10]:
joined_df.show(3)

+-----------+---------------+--------------------+----------------+---------------------+-------------------+-------------+-------------------+-------------------+-----------+------------------+--------------------+------------------+
|MailPieceID|ServiceTypeCode|StartTheClockZipCode|StopTheClockDate|ScheduledDeliveryDate|StopTheClockZipCode|MailClassCode|  min_scan_datetime|  max_scan_datetime|total_scans|Distinct_zip_scans|Distinct_event_scans|time_delta_minutes|
+-----------+---------------+--------------------+----------------+---------------------+-------------------+-------------+-------------------+-------------------+-----------+------------------+--------------------+------------------+
|85660637209|           020 |               42240|      2024-01-22|           2024-01-22|              98230|           G0|2023-09-26 00:03:03|2024-01-22 12:01:21|         16|                 5|                  12|            169920|
|85684177804|           020 |               42141|      2024

In [11]:
from pyspark.sql.functions import col, to_date

joined_df = joined_df.withColumn("min_date", to_date(col("min_scan_datetime")))
joined_df = joined_df.withColumn("max_date", to_date(col("max_scan_datetime")))

joined_df.show(3)

+-----------+---------------+--------------------+----------------+---------------------+-------------------+-------------+-------------------+-------------------+-----------+------------------+--------------------+------------------+----------+----------+
|MailPieceID|ServiceTypeCode|StartTheClockZipCode|StopTheClockDate|ScheduledDeliveryDate|StopTheClockZipCode|MailClassCode|  min_scan_datetime|  max_scan_datetime|total_scans|Distinct_zip_scans|Distinct_event_scans|time_delta_minutes|  min_date|  max_date|
+-----------+---------------+--------------------+----------------+---------------------+-------------------+-------------+-------------------+-------------------+-----------+------------------+--------------------+------------------+----------+----------+
|85660637209|           020 |               42240|      2024-01-22|           2024-01-22|              98230|           G0|2023-09-26 00:03:03|2024-01-22 12:01:21|         16|                 5|                  12|            16

In [12]:
from pyspark.sql.functions import when, col, expr
joined_df = joined_df.withColumn("late", when(col("StopTheClockDate") > col("ScheduledDeliveryDate"), 1).otherwise(0))

In [13]:
joined_df.show(3)

+-----------+---------------+--------------------+----------------+---------------------+-------------------+-------------+-------------------+-------------------+-----------+------------------+--------------------+------------------+----------+----------+----+
|MailPieceID|ServiceTypeCode|StartTheClockZipCode|StopTheClockDate|ScheduledDeliveryDate|StopTheClockZipCode|MailClassCode|  min_scan_datetime|  max_scan_datetime|total_scans|Distinct_zip_scans|Distinct_event_scans|time_delta_minutes|  min_date|  max_date|late|
+-----------+---------------+--------------------+----------------+---------------------+-------------------+-------------+-------------------+-------------------+-----------+------------------+--------------------+------------------+----------+----------+----+
|85660637209|           020 |               42240|      2024-01-22|           2024-01-22|              98230|           G0|2023-09-26 00:03:03|2024-01-22 12:01:21|         16|                 5|                  12

In [15]:
joined_df.printSchema()

root
 |-- MailPieceID: string (nullable = true)
 |-- ServiceTypeCode: string (nullable = true)
 |-- StartTheClockZipCode: string (nullable = true)
 |-- StopTheClockDate: date (nullable = true)
 |-- ScheduledDeliveryDate: date (nullable = true)
 |-- StopTheClockZipCode: string (nullable = true)
 |-- MailClassCode: string (nullable = true)
 |-- min_scan_datetime: timestamp (nullable = true)
 |-- max_scan_datetime: timestamp (nullable = true)
 |-- total_scans: long (nullable = true)
 |-- Distinct_zip_scans: long (nullable = true)
 |-- Distinct_event_scans: long (nullable = true)
 |-- time_delta_minutes: integer (nullable = true)
 |-- min_date: date (nullable = true)
 |-- max_date: date (nullable = true)
 |-- late: integer (nullable = false)



In [16]:
#################################################################################################

In [16]:
joined_df.write.mode('overwrite').parquet("/home/pk/DAEN690/PKG_Origin_cleaned.parquet")
print("done")

done


In [17]:
df = spark.read.parquet("/home/pk/DAEN690/PKG_Origin_cleaned.parquet")

In [18]:
#1115279 conts before join
df.show(3)
df.count()

+-----------+---------------+--------------------+----------------+---------------------+-------------------+-------------+-------------------+-------------------+-----------+------------------+--------------------+------------------+----------+----------+----+
|MailPieceID|ServiceTypeCode|StartTheClockZipCode|StopTheClockDate|ScheduledDeliveryDate|StopTheClockZipCode|MailClassCode|  min_scan_datetime|  max_scan_datetime|total_scans|Distinct_zip_scans|Distinct_event_scans|time_delta_minutes|  min_date|  max_date|late|
+-----------+---------------+--------------------+----------------+---------------------+-------------------+-------------+-------------------+-------------------+-----------+------------------+--------------------+------------------+----------+----------+----+
|85660637209|           020 |               42240|      2024-01-22|           2024-01-22|              98230|           G0|2023-09-26 00:03:03|2024-01-22 12:01:21|         16|                 5|                  12

1115279

In [19]:
dict_rename = {"StartTheClockZipCode": "Origin_zip", "StopTheClockZipCode": "destination_zip"}

for old_name, new_name in dict_rename.items():
    df = df.withColumnRenamed(old_name, new_name)

df.show()

+-----------+---------------+----------+----------------+---------------------+---------------+-------------+-------------------+-------------------+-----------+------------------+--------------------+------------------+----------+----------+----+
|MailPieceID|ServiceTypeCode|Origin_zip|StopTheClockDate|ScheduledDeliveryDate|destination_zip|MailClassCode|  min_scan_datetime|  max_scan_datetime|total_scans|Distinct_zip_scans|Distinct_event_scans|time_delta_minutes|  min_date|  max_date|late|
+-----------+---------------+----------+----------------+---------------------+---------------+-------------+-------------------+-------------------+-----------+------------------+--------------------+------------------+----------+----------+----+
|85660637209|           020 |     42240|      2024-01-22|           2024-01-22|          98230|           G0|2023-09-26 00:03:03|2024-01-22 12:01:21|         16|                 5|                  12|            169920|2023-09-26|2024-01-22|   0|
|8568417

In [20]:
columns_to_drop = ["StopTheClockDate", "min_scan_datetime", "max_scan_datetime"]

df = df.drop(*columns_to_drop)

df.show()

+-----------+---------------+----------+---------------------+---------------+-------------+-----------+------------------+--------------------+------------------+----------+----------+----+
|MailPieceID|ServiceTypeCode|Origin_zip|ScheduledDeliveryDate|destination_zip|MailClassCode|total_scans|Distinct_zip_scans|Distinct_event_scans|time_delta_minutes|  min_date|  max_date|late|
+-----------+---------------+----------+---------------------+---------------+-------------+-----------+------------------+--------------------+------------------+----------+----------+----+
|85660637209|           020 |     42240|           2024-01-22|          98230|           G0|         16|                 5|                  12|            169920|2023-09-26|2024-01-22|   0|
|85684177804|           020 |     42141|           2024-01-12|          91715|           G0|         19|                 7|                  13|            164160|2023-09-26|2024-01-18|   1|
|85710348611|           020 |     37062|     

In [21]:
from pyspark.sql.functions import date_format
dict_rename = {"min_date": "Origin_date", "max_date": "destination_date"}

for old_name, new_name in dict_rename.items():
    df = df.withColumnRenamed(old_name, new_name)
    
df = df.withColumn("YEARMONTH_Ori", date_format(col("Origin_date"), "yyyyMM"))
df = df.withColumn("YEARMONTH_Des", date_format(col("destination_date"), "yyyyMM"))

df.show()

+-----------+---------------+----------+---------------------+---------------+-------------+-----------+------------------+--------------------+------------------+-----------+----------------+----+-------------+-------------+
|MailPieceID|ServiceTypeCode|Origin_zip|ScheduledDeliveryDate|destination_zip|MailClassCode|total_scans|Distinct_zip_scans|Distinct_event_scans|time_delta_minutes|Origin_date|destination_date|late|YEARMONTH_Ori|YEARMONTH_Des|
+-----------+---------------+----------+---------------------+---------------+-------------+-----------+------------------+--------------------+------------------+-----------+----------------+----+-------------+-------------+
|85660637209|           020 |     42240|           2024-01-22|          98230|           G0|         16|                 5|                  12|            169920| 2023-09-26|      2024-01-22|   0|       202309|       202401|
|85684177804|           020 |     42141|           2024-01-12|          91715|           G0|    

In [22]:
df.write.mode('overwrite').parquet("/home/pk/DAEN690/PKG_Origin_cleaned_YM.parquet") 
print("done")

done


Get LAT LON for zipcodes in Data


In [23]:
from pyspark.sql import functions as F

df_col1 = df.select(F.col("Origin_zip").alias("zipcode"))
df_col2 = df.select(F.col("destination_zip").alias("zipcode"))

union_df = df_col1.union(df_col2)

In [24]:
unique_values_df = union_df.distinct()
unique_values_df.count()

32043

In [25]:
unique_values_df.write.mode('overwrite').parquet("/home/pk/DAEN690/PKG_zip.parquet")
print("done")

done


In [26]:
df_zip = spark.read.parquet("/home/pk/DAEN690/PKG_zip.parquet")
df_zip.show(3)
df_zip.count()

+-------+
|zipcode|
+-------+
|  37160|
|  37228|
|  42134|
+-------+
only showing top 3 rows



32043

In [27]:
csv_file_path = "/home/pk/DAEN690/2018_zip_lat_lon.csv"
zip_lat_lon_df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

zip_lat_lon_df.show()

+---+---------+----------+
|ZIP|      LAT|       LNG|
+---+---------+----------+
|601|18.180555|-66.749961|
|602|18.361945|-67.175597|
|603|18.455183|-67.119887|
|606|18.158327|-66.932928|
|610|18.295366|-67.125135|
|612|18.402253|-66.711397|
|616|18.420412|-66.671979|
|617|18.447538|-66.557681|
|622|17.991245|-67.153993|
|623|18.083361|-67.153897|
|624|18.064919|-66.716683|
|627|  18.4126|-66.863926|
|631|18.190607|-66.832041|
|637|18.076713|-66.947389|
|638|18.295913|-66.515588|
|641|18.263097|-66.712963|
|646| 18.43315|-66.285875|
|647|17.963613|-66.947127|
|650|18.349416|-66.578079|
|652| 18.44844| -66.59411|
+---+---------+----------+
only showing top 20 rows



In [28]:
# Join on the zip code column
joined_df = df_zip.join(zip_lat_lon_df, df_zip["zipcode"] == zip_lat_lon_df["ZIP"], "inner")

In [29]:
columns_to_drop = ["ZIP"]

joined_df = joined_df.drop(*columns_to_drop)
joined_df.show(3)
joined_df.count()

+-------+---------+----------+
|zipcode|      LAT|       LNG|
+-------+---------+----------+
|  00601|18.180555|-66.749961|
|  00602|18.361945|-67.175597|
|  00603|18.455183|-67.119887|
+-------+---------+----------+
only showing top 3 rows



29090

In [31]:
joined_df.write.mode('overwrite').parquet("/home/pk/DAEN690/zip_latlon.parquet") 
print("done")

done
